This notebook uses the CSVs created with the EdNet - 1 - Feature engineering notebook. Make sure to run the feature engineering code before continuing with this notebook. Additionally, this notebook uses the non-standard library [PyTorch](https://pytorch.org/), which you may need to install before proceeding.

### Import libraries

In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils as torch_utils
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

### Define function to create final question features based on users in training set

In [ ]:
def create_question_features(x):
    x_new = x.sort_values('timestamp')
    
    q_accuracy = []
    part_accuracy = []
    user_excess_correct = []
    
    q_dict = {}
    part_dict = {}
    user_dict = {}
    for part, q_id, u_id, correct in zip(x_new['part'], x_new['question_id'], x_new['user_id'], x_new['correct_response']):
        # Calculate excess correct first to avoid contamination
        excess_correct = 0
        if q_id in q_dict:
            avg_q_acc = q_dict[q_id]['n_correct']/q_dict[q_id]['n_ans']
            excess_correct = correct - avg_q_acc
        elif part in part_dict:
            avg_p_acc = part_dict[part]['n_correct']/part_dict[part]['n_ans']
            excess_correct = correct - avg_p_acc
        else:
            excess_correct = correct - 0.5# default
        
        if q_id in q_dict:
            q_accuracy.append(q_dict[q_id]['n_correct']/q_dict[q_id]['n_ans'])
            q_dict[q_id]['n_ans'] += 1
            q_dict[q_id]['n_correct'] += correct
        else:
            q_accuracy.append(np.nan)
            q_dict[q_id] = {'n_ans': 1, 'n_correct': correct}
            
        if part in part_dict:
            part_accuracy.append(part_dict[part]['n_correct']/part_dict[part]['n_ans'])
            part_dict[part]['n_ans'] += 1
            part_dict[part]['n_correct'] += correct
        else:
            part_accuracy.append(np.nan)
            part_dict[part] = {'n_ans': 1, 'n_correct': correct}
            
        if u_id in user_dict:
            avg_excess_correct = user_dict[u_id]['sum_excess_correct'] / user_dict[u_id]['n_ans']
            user_excess_correct.append(avg_excess_correct)
            
            user_dict[u_id]['n_ans'] += 1
            user_dict[u_id]['sum_excess_correct'] += excess_correct
        else:
            user_excess_correct.append(np.nan)
            user_dict[u_id] = {'n_ans': 1, 'sum_excess_correct': excess_correct}
            
    x_new['q_acc'] = q_accuracy
    x_new['part_acc'] = part_accuracy
    x_new['usr_excess_correct'] = user_excess_correct
    
    return x_new

### Define functions to split data into train-test and into local clients

In [ ]:
def X_y_from_df(df):
    X = df.drop(columns=['timestamp', 'solving_id', 'question_id', 'elapsed_time',
                                 'user_id', 'part', 'correct_response']).fillna(0).to_numpy()
    y = df['correct_response'].to_numpy().ravel()
    
    X_torch = torch.tensor(X, dtype=torch.float32)
    y_torch = torch.tensor(y, dtype=torch.int64)
    
    return X_torch, y_torch

In [ ]:
def split_into_holdout_and_clients(df, n_clients, seed = 42):
    np.random.seed(seed)
    
    uids = list(df['user_id'].unique())
    np.random.shuffle(uids)
    
    n_holdout_uids = int(len(uids) * 0.2)
    holdout_uids = uids[0:n_holdout_uids]
    local_uids = uids[n_holdout_uids:]
    
    df_holdout = create_question_features(df.loc[df['user_id'].isin(holdout_uids)])
    X_holdout, y_holdout = X_y_from_df(df_holdout)
    
    clients_uids = np.array_split(local_uids, n_clients)
    
    clients = []
    for client_uids in clients_uids:
        df_client = create_question_features(df.loc[df['user_id'].isin(client_uids)])
        
        X_client, y_client = X_y_from_df(df_client)

        clients.append((X_client, y_client))

    return X_holdout, y_holdout, clients

### Define federated learning functions

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(NeuralNetwork, self).__init__()
        self.input_layer    = nn.Linear(in_dim,16)
        self.hidden_layer1  = nn.Linear(16,8)
        self.output_layer   = nn.Linear(8,out_dim)
        self.relu = nn.ReLU()
    
    def forward(self,x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.output_layer(out)
        return out
    
def federated_averaging(global_model, local_models, num_clients, client_sizes):
    dataset_size = sum(client_sizes)
    
    for param_global, params_local in zip(global_model.parameters(), zip(*[model.parameters() for model in local_models])):
        weighted_sum = torch.zeros_like(param_global.data)
        for client_params, client_size in zip(params_local, client_sizes):
            client_weight = client_size / dataset_size
            weighted_sum += client_weight * client_params.data
        
        param_global.data = weighted_sum
        
def train_local_model(model, dataloader, optimizer, loss_fn, epochs, device):
    for epoch in range(epochs):
        model.train()
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            prediction = model(data)
            loss = loss_fn(prediction, target)
            loss.backward()
            optimizer.step()
            
def validate_local_model(model, dataloader, loss_fn):
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for data, target in dataloader:
            prediction = model(data)
            total_loss += loss_fn(prediction, target).item()
            _, predicted_labels = torch.max(prediction, 1)
            correct_predictions += (predicted_labels == target).sum().item()
            total_samples += len(target)

    average_loss = total_loss / len(dataloader)
    accuracy = correct_predictions / total_samples

    return average_loss, accuracy

In [ ]:
def federated_learning(clients, input_dim, output_dim, loss_fn, lr, optim_str, num_clients, num_rounds, num_epochs, batch_size):
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device = torch.device("cpu")
    
    global_model = NeuralNetwork(input_dim, output_dim).to(device)
    
    client_sizes = [len(client) for client in clients]
    for r in range(num_rounds):
        local_models = []
        local_optimizers = []
        local_dataloaders = []
        
        for i in range(num_clients):
            local_model = NeuralNetwork(input_dim, output_dim).to(device)
            local_model.load_state_dict(global_model.state_dict())
            local_optimizer = optim.SGD(local_model.parameters(), lr=lr)
            if optim_str == 'adam':
                local_optimizer = optim.Adam(local_model.parameters(), lr=lr)
            
            X, y = clients[i]
            X, y = X.to(device), y.to(device)
            local_dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X, y),
                                                           batch_size=batch_size, shuffle=True)
            train_local_model(local_model, local_dataloader, local_optimizer, loss_fn, num_epochs, device)
            
            local_models.append(local_model)
            local_optimizers.append(local_optimizer)
            local_dataloaders.append(local_dataloader)
            
        federated_averaging(global_model, local_models, num_clients, client_sizes)
        
        if r % 5 == 0:
            for i in range(num_clients):
                X, y = clients[i]
                X, y = X.to(device), y.to(device)
                local_dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X, y),
                                                               batch_size=batch_size, shuffle=False)
                val_loss, val_accuracy = validate_local_model(local_models[i], local_dataloader, loss_fn)
                print(f"Client {i+1} - Round {r + 1}/{num_rounds}, Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}")
            
    return global_model

### Read data and train federated learning model

For this example we use N_CLIENTS = 10. Results for other numbers of local clients can easily be obtained by changing the value of N_CLIENTS.

In [ ]:
df = pd.read_csv('data/ednet_features_10000_users.csv')

loss_fn = nn.CrossEntropyLoss()
lr = 0.02
optimizer = 'adam'

N_CLIENTS = 10
N_ROUNDS = 20
N_EPOCHS = 2
BATCH_SIZE = 128

acc_fed, f1_fed, auc_fed = [], [], []
for i in range(10):
    X_holdout, y_holdout, torch_clients = split_into_holdout_and_clients(df, N_CLIENTS, seed = i)
    
    input_dim = X_holdout.shape[1]
    output_dim = len(np.unique(y_holdout))
    
    global_model = federated_learning(torch_clients, input_dim, output_dim, loss_fn, lr, optimizer, N_CLIENTS, N_ROUNDS, N_EPOCHS, BATCH_SIZE)
    
    global_model.eval()
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device = torch.device("cpu")
    with torch.no_grad():
        predictions = global_model(X_holdout.to(device))
        
    _, y_pred = torch.max(predictions, dim=1)
    y_probs = F.softmax(predictions, dim=1)[:, 1]
    
    y_pred = y_pred.cpu().numpy()
    y_probs = y_probs.cpu().numpy()
    
    acc_fed.append(accuracy_score(y_holdout, y_pred))
    f1_fed.append(f1_score(y_holdout, y_pred))
    auc_fed.append(roc_auc_score(y_holdout, y_probs))

### Store results as CSV

In [ ]:
df = pd.DataFrame({'acc': acc_fed, 'f1': f1_fed, 'auc': auc_fed})

df.to_csv('ednet_federated_10000users_10clients.csv', index = False)